## Panimula

Tatalakayin sa araling ito ang:
- Ano ang function calling at mga gamit nito
- Paano gumawa ng function call gamit ang Azure OpenAI
- Paano isama ang function call sa isang application

## Mga Layunin sa Pagkatuto

Pagkatapos ng araling ito, malalaman mo kung paano at maiintindihan mo ang:

- Layunin ng paggamit ng function calling
- Pagsasaayos ng Function Call gamit ang Azure Open AI Service
- Pagdidisenyo ng epektibong function calls para sa iyong application at mga gamit nito


## Pag-unawa sa Function Calls

Para sa leksyong ito, gusto nating gumawa ng isang feature para sa ating education startup na magpapahintulot sa mga user na gumamit ng chatbot para maghanap ng mga technical na kurso. Magrerekomenda tayo ng mga kurso na akma sa kanilang kasanayan, kasalukuyang tungkulin, at teknolohiyang interesado sila.

Para magawa ito, gagamit tayo ng kombinasyon ng:
 - `Azure Open AI` para gumawa ng chat experience para sa user
 - `Microsoft Learn Catalog API` para tulungan ang mga user na makahanap ng mga kurso base sa kanilang hinihiling
 - `Function Calling` para kunin ang query ng user at ipadala ito sa isang function para mag-request sa API.

Para makapagsimula, tingnan muna natin kung bakit natin gustong gumamit ng function calling:

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # kumuha ng bagong response mula sa GPT kung saan makikita nito ang sagot ng function

print(second_response.choices[0].message)


### Bakit Mahalaga ang Function Calling

Kung nakatapos ka na ng ibang aralin sa kursong ito, malamang alam mo na kung gaano ka-powerful ang paggamit ng Large Language Models (LLMs). Sana napansin mo rin ang ilang limitasyon ng mga ito.

Ang Function Calling ay isang tampok ng Azure Open AI Service na tumutulong malampasan ang mga sumusunod na limitasyon:
1) Pare-parehong format ng sagot
2) Kakayahang gumamit ng datos mula sa ibang sources ng isang application sa chat context

Bago pa man ang function calling, ang mga sagot mula sa LLM ay kadalasang walang istraktura at hindi pare-pareho. Kailangan pang magsulat ng mga developer ng kumplikadong validation code para masigurong kaya nilang hawakan ang bawat uri ng sagot.

Hindi rin makakuha ng sagot ang mga user sa mga tanong tulad ng "Ano ang kasalukuyang lagay ng panahon sa Stockholm?". Ito ay dahil limitado ang mga modelo sa panahon kung kailan sila na-train.

Tingnan natin ang halimbawa sa ibaba na nagpapakita ng problemang ito:

Halimbawa, gusto nating gumawa ng database ng impormasyon ng mga estudyante para makapag-suggest tayo ng tamang kurso para sa kanila. Sa ibaba, may dalawang paglalarawan ng mga estudyante na halos magkapareho ang laman ng datos.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Gusto naming ipadala ito sa isang LLM para i-parse ang data. Maaari itong magamit sa aming application para ipadala ito sa isang API o i-store sa isang database.

Gumawa tayo ng dalawang magkaparehong prompt na magbibigay ng instruksyon sa LLM kung anong impormasyon ang gusto naming makuha:


Gusto naming ipadala ito sa isang LLM upang suriin ang mga bahagi na mahalaga sa aming produkto. Kaya maaari kaming gumawa ng dalawang magkaparehong prompt upang bigyan ng tagubilin ang LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Pagkatapos gawin ang dalawang prompt na ito, ipapadala natin ang mga ito sa LLM gamit ang `openai.ChatCompletion`. Iniimbak natin ang prompt sa variable na `messages` at itinatakda ang role bilang `user`. Ito ay upang tularan ang isang mensahe mula sa isang user na isinusulat sa isang chatbot.


In [ ]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-07-01-preview"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Kahit na pareho ang mga prompt at magkahawig ang mga deskripsyon, maaari tayong makakuha ng iba’t ibang format ng property na `Grades`.

Kung paulit-ulit mong patakbuhin ang cell sa itaas, maaaring maging `3.7` o `3.7 GPA` ang format.

Ito ay dahil ang LLM ay tumatanggap ng hindi istrakturadong datos mula sa isinulat na prompt at nagbabalik din ng hindi istrakturadong datos. Kailangan nating magkaroon ng istrakturadong format para alam natin kung ano ang aasahan kapag iniimbak o ginagamit ang datos na ito.

Sa pamamagitan ng paggamit ng functional calling, masisiguro natin na pabalik ay istrakturadong datos ang matatanggap natin. Kapag gumagamit ng function calling, hindi talaga tumatawag o nagpapatakbo ng anumang function ang LLM. Sa halip, gumagawa tayo ng isang istraktura na susundan ng LLM para sa mga sagot nito. Ginagamit natin ang mga istrakturadong sagot na ito para malaman kung anong function ang dapat patakbuhin sa ating mga application.


![Function Calling Flow Diagram](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.tl.png)


### Mga Halimbawa ng Paggamit ng function calls

**Pagtawag ng Panlabas na Mga Tool**  
Mahusay ang mga chatbot sa pagbibigay ng sagot sa mga tanong ng mga user. Sa pamamagitan ng function calling, magagamit ng mga chatbot ang mga mensahe ng user para tapusin ang ilang gawain. Halimbawa, maaaring sabihin ng isang estudyante sa chatbot na "Magpadala ng email sa guro ko na kailangan ko pa ng tulong sa subject na ito." Maaari nitong gamitin ang function na `send_email(to: string, body: string)`

**Paglikha ng API o Database Queries**  
Maaaring maghanap ng impormasyon ang mga user gamit ang natural na wika na iko-convert sa isang formatted na query o API request. Halimbawa nito ay isang guro na nagtatanong, "Sino-sino ang mga estudyanteng nakatapos ng huling assignment?" na maaaring gumamit ng function na `get_completed(student_name: string, assignment: int, current_status: string)`

**Paglikha ng Structured Data**  
Maaaring kumuha ang mga user ng isang block ng text o CSV at gamitin ang LLM para kunin ang mahahalagang impormasyon mula rito. Halimbawa, maaaring gawing AI flash cards ng isang estudyante ang isang Wikipedia article tungkol sa mga kasunduan sa kapayapaan. Magagawa ito gamit ang function na `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Paglikha ng Iyong Unang Function Call

Ang proseso ng paggawa ng function call ay binubuo ng 3 pangunahing hakbang:
1. Tawagin ang Chat Completions API gamit ang listahan ng iyong mga function at isang mensahe mula sa user
2. Basahin ang sagot ng modelo upang magsagawa ng aksyon, halimbawa magpatakbo ng function o API Call
3. Gumawa ng panibagong tawag sa Chat Completions API gamit ang sagot mula sa iyong function upang magamit ang impormasyong iyon sa paggawa ng sagot para sa user.


### Mga Elemento ng Pagtawag ng Function

#### Input ng User

Ang unang hakbang ay gumawa ng mensahe mula sa user. Maaari itong itakda nang dynamic gamit ang halaga mula sa isang text input o maaari kang magtakda ng halaga dito. Kung ito ang unang beses mong gagamitin ang Chat Completions API, kailangan nating tukuyin ang `role` at ang `content` ng mensahe.

Ang `role` ay maaaring `system` (gumagawa ng mga patakaran), `assistant` (ang modelo), o `user` (ang end-user). Para sa function calling, itatakda natin ito bilang `user` at maglalagay ng halimbawa ng tanong.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Paglikha ng mga function.

Susunod, magde-define tayo ng isang function at ang mga parameter nito. Gagamit tayo dito ng isang function na tinatawag na `search_courses` pero maaari kang gumawa ng maraming function.

**Mahalaga**: Kasama ang mga function sa system message na ipinapadala sa LLM at isasama ito sa kabuuang bilang ng tokens na maaari mong gamitin.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Mga Kahulugan**

`name` - Ang pangalan ng function na gusto nating tawagin.

`description` - Ito ang paliwanag kung paano gumagana ang function. Dito mahalagang maging tiyak at malinaw.

`parameters` - Isang listahan ng mga halaga at format na gusto mong ilabas ng model sa sagot nito.

`type` - Ang uri ng data kung saan itatago ang mga properties.

`properties` - Listahan ng mga partikular na halaga na gagamitin ng model para sa sagot nito.

`name` - Ang pangalan ng property na gagamitin ng model sa naka-format nitong sagot.

`type` - Ang uri ng data ng property na ito.

`description` - Paliwanag ng partikular na property.

**Opsyonal**

`required` - kinakailangang property para makumpleto ang function call


### Paggawa ng function call
Pagkatapos magtakda ng function, kailangan na natin itong isama sa pagtawag sa Chat Completion API. Ginagawa natin ito sa pamamagitan ng pagdagdag ng `functions` sa request. Sa kasong ito, `functions=functions`.

Mayroon ding opsyon na itakda ang `function_call` sa `auto`. Ibig sabihin nito, hahayaan natin ang LLM na pumili kung aling function ang tatawagin base sa mensahe ng user, sa halip na tayo mismo ang magtakda.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Ngayon, tingnan natin ang tugon at kung paano ito naka-format:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Makikita mo na tinawag ang pangalan ng function at mula sa mensahe ng user, nakuha ng LLM ang mga datos na kailangan para mapunan ang arguments ng function.


## 3. Pagsasama ng Function Calls sa isang Aplikasyon.

Matapos nating masubukan ang na-format na sagot mula sa LLM, maaari na natin itong isama sa isang aplikasyon.

### Pamamahala ng Daloy

Para maisama ito sa ating aplikasyon, sundin natin ang mga hakbang na ito:

Una, gawin natin ang tawag sa Open AI services at itago ang mensahe sa isang variable na tinatawag na `response_message`.


In [ ]:
response_message = response.choices[0].message

Ngayon ay itatala natin ang function na tatawag sa Microsoft Learn API upang makuha ang listahan ng mga kurso:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Bilang isang pinakamainam na gawain, titignan muna natin kung gusto ng modelo na tumawag ng isang function. Pagkatapos nito, gagawa tayo ng isa sa mga available na function at itutugma ito sa function na tinatawag.  
Pagkatapos, kukunin natin ang mga argumento ng function at itutugma sa mga argumento mula sa LLM.

Sa huli, idadagdag natin ang mensahe ng function call at ang mga halagang ibinalik ng `search_courses` na mensahe. Bibigyan nito ang LLM ng lahat ng impormasyong kailangan nito para
tumugon sa user gamit ang natural na wika.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Hamon sa Pag-code

Magaling! Para ipagpatuloy ang iyong pagkatuto tungkol sa Azure Open AI Function Calling, maaari mong subukan ang: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - Magdagdag ng mas maraming parameter sa function na makakatulong sa mga mag-aaral na makahanap ng mas maraming kurso. Makikita mo ang mga available na API parameter dito:
 - Gumawa ng isa pang function call na kumukuha ng karagdagang impormasyon mula sa mag-aaral tulad ng kanilang katutubong wika
 - Gumawa ng error handling kapag ang function call at/o API call ay hindi nagbalik ng angkop na mga kurso



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama’t nagsusumikap kami para sa kawastuhan, pakatandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi eksaktong pagsasalin. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring lumitaw mula sa paggamit ng pagsasaling ito.
